Tools, helpful functions for development and debugging
====================



In [3]:
from sage.algebras.flag_algebras import *

In [72]:
sys.setprofile(None)

def tracefunc(frame, event, arg, indent=[0]):
    if event == "call":
        indent[0] += 2
        module_name = inspect.getmodule(frame.f_code).__name__ if inspect.getmodule(frame.f_code) else "unknown"
        class_name = ""
        try:
            class_name = inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True)[-1][0][0].__name__
        except:
            if 'self' in frame.f_locals:
                class_name = str(inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True))
            else:
                class_name = ""
        function_name = frame.f_code.co_name
        tst = module_name + function_name
        if "IPython" in tst or "enum" in tst or "typing" in tst or "traitlets" in tst or "util" in tst or "ipykernel" in tst:
            pass
        else:
            if "unknown" in tst or "tokenize" in tst or "logging" in tst or "json" in tst or "preparse" in tst or indent[0]<=0:
                pass
            else:
                print("-" * indent[0] + "> call function", function_name, "in", class_name, "from", module_name)
    elif event == "return":
        indent[0] -= 2
    return tracefunc

#sys.setprofile(tracefunc)

#asd = sum([G(0)])

In [19]:
#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#for debug printing values with their variable names
def debug(*args):
    return
    import inspect
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = s.split('(')[1].split(')')[0].split(',')
    names = [name.strip() for name in r]
    
    for name, value in zip(names, args):
        if isinstance(value, str):
            print(value)
        else:
            print(f"{name}: {value}")


In [2]:
def test_identify(n, ftype_points, edges, C0, C1):
    return colored_identify(2, [[0, 1]], n, ftype_points, edges=edges, C0=C0, C1=C1)

#same for the generator. k=2 and order_partition=[[0, 1]].
def test_generate(n):
    return colored_generate(2, [[0, 1]], n)

TTT = CombinatorialTheory("2UnOrdColGraph", test_generate, test_identify, edges=2, C0=1, C1=1)

tri_000 = TTT(3, edges=[[0, 1], [0, 2], [1, 2]], C0=[[0], [1], [2]])
tri_001 = TTT(3, edges=[[0, 1], [0, 2], [1, 2]], C0=[[0], [1]], C1=[[2]])
TTT.exclude([tri_000, tri_001])

Bp = TTT(2, edges=[[0, 1]], C0=[[0], [1]], ftype=[0])
Gp = TTT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]], ftype=[0])

G = TTT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]])
B = TTT(2, edges=[[0, 1]], C0=[[0], [1]])
M = TTT(2, C0=[[0]], C1=[[1]])

optim = TTT.blowup_construction(4, 2, edges=[[0, 1]], C0=[[0]], C1=[[1]], symmetric=True, symbolic=True)
ssoptim = optim.set_sum()
der_optims = ssoptim.derivatives([1/2])

loglevel=1
TTT.optimize_problem(B - M, 4, positives=[Gp-Bp, G-1/3], certificate=False, construction=der_optims)

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 31.39it/s]


Base flags generated
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 8, 6, 6]


done with mult table for Ftype on 2 points with edges=[[0, 1]], C0=[[0]], C1=[[1]]: : 4it [00:00, 16.21it/s]  


Tables constructed
Done with constraint for 
Flag Algebra Element over Rational Field
0  - Flag on 2 points, ftype from [0] with edges=[], C0=[[0], [1]], C1=[]
0  - Flag on 2 points, ftype from [0] with edges=[], C0=[[0]], C1=[[1]]
-1 - Flag on 2 points, ftype from [0] with edges=[[0, 1]], C0=[[0], [1]], C1=[]
1  - Flag on 2 points, ftype from [0] with edges=[[0, 1]], C0=[[0]], C1=[[1]]



TypeError: unsupported operand parent(s) for *: 'Sparse vector space of dimension 1 over Real Double Field' and 'Sparse vector space of dimension 4 over Rational Field'

In [20]:
def optimize_problem(self, target_element, target_size, maximize=True, positives=None, \
                     construction=None, certificate=False, exact=False, ftype_pairs=None):
    
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    import io
    import time

    #
    # initial setup
    #

    if target_size not in self.sizes():
        raise ValueError("For theory {}, size {} is not allowed.".format(self._name, target_size))

    base_flags = self.generate_flags(target_size)
    print("Base flags generated, their number is {}".format(len(base_flags)))
    mult = -1 if maximize else 1
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    
    #
    # create the table data
    #

    if ftype_pairs==None:
        plausible_sizes = []
        for fs in self.sizes():
            if fs>=target_size:
                break
            if fs==0:
                continue
            plausible_sizes.append(fs)
        ftype_pairs = []
        for fs, ns in itertools.combinations(plausible_sizes, r=int(2)):
            if 2*ns-fs <= target_size:
                kk = ns-fs
                found = False
                for ii, (bfs, bns) in enumerate(ftype_pairs):
                    if bns-bfs==kk:
                        found = True
                        if ns>bns:
                            ftype_pairs[ii]=(fs, ns)
                        break
                if not found:
                    ftype_pairs.append((fs, ns))
    else:
        for xx, yy in ftype_pairs:
            if xx not in self.sizes() or yy not in self.sizes():
                raise ValueError("The ftype pairs contain sizes that are not allowed.")

    ftype_data = []
    for fs, ns in ftype_pairs:
        ftype_flags = self.generate_flags(fs)
        ftypes = [flag.subflag([], ftype_points=list(range(fs))) for flag in ftype_flags]
        for xx in ftypes:
            ftype_data.append((ns, xx, target_size))

    print("The relevant ftypes are constructed, their number is {}".format(len(ftype_data)))

    flags = [self.generate_flags(dat[0], dat[1]) for dat in ftype_data]
    flag_sizes = [len(xx) for xx in flags]

    print("Block sizes before symmetric/asymmetric change is applied: {}".format(flag_sizes))

    sym_asym_mats = [self.sym_asym_bases(dat[0], dat[1]) for dat in ftype_data]

    table_constructor = {}
    for ii, dat in (pbar := tqdm(enumerate(ftype_data))):
        ns, ftype, target_size = dat
        #pre-calculate the table here
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        if table==None:
            pbar.set_description("Structures with size {} and {} had singular multiplication table!".format(ns, ftype))
            continue
        sym_base, asym_base = sym_asym_mats[ii]
        bases = []
        if sym_base.nrows()!=0:
            bases.append(sym_base)
        if asym_base.nrows()!=0:
            bases.append(asym_base)
        table_constructor[dat] = bases
        pbar.set_description("done with mult table for {}".format(ftype))
    print("Tables constructed")

    sdp_data = self._tables_to_sdp_data(table_constructor)

    #
    # add constraints data
    #

    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
            print("Done with constraint for \n{}\n".format(fv))
        constraints_vals = [0]*len(constraints_flags)
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    positives_list_exact = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    positives_matrix_exact = matrix(positives_list_exact)

    sdp_data = self._constraints_to_sdp_data(len(base_flags), constraints_vals, positives_list_exact, sdp_data)

    #
    # if no y value provided, run the optimizer first, only to get the y values
    #
    if construction==None:
        mat_inds, mat_vals, block_sizes = sdp_data
        print("Running sdp without construction. Used block sizes are {}".format(block_sizes))


        time.sleep(float(0.1))
        initial_sol = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
        time.sleep(float(0.1))

        if (not exact):
            res = initial_sol['primal'] * (-1 if maximize else 1)
            return res if (not certificate) else (res, initial_sol)

        phi_vector_original = initial_sol['y']
        phi_vector_rounded = _round_adaptive(initial_sol['y'], one_vector.values())
        alg = FlagAlgebra(QQ, self)
        print("rounded phi vector is: \n{}".format(alg(target_size, phi_vector_rounded)))
        phi_vectors_exact = [phi_vector_rounded]
    else:
        if isinstance(construction, FlagAlgebraElement):
            phi_vectors_exact = [construction.values()]
        else:
            phi_vectors_exact = [xx.values() for xx in construction]
    
    #
    # adjust the table to consider the kernel from y_rounded
    #

    print("Adjusting table with kernels from construction")
    table_constructor = self._adjust_table_phi(table_constructor, phi_vectors_exact)

    sdp_data = self._tables_to_sdp_data(table_constructor)
    sdp_data = self._constraints_to_sdp_data(len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    mat_inds, mat_vals, block_sizes = sdp_data

    print("running SDP after kernel correction. Used with block sizes are {}".format(block_sizes))

    time.sleep(float(0.1))
    final_sdp = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
    time.sleep(float(0.1))

    if (not exact):
        res = final_sdp['primal'] * (-1 if maximize else 1)
        return res if (not certificate) else (res, final_sdp)
    
    print("Starting the rounding of the result.")
    rounded = _round_sdp_solution(self, final_sdp, table_constructor, block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact)
    
    res = rounded[0] * (-1 if maximize else 1)
    return res if (not certificate) else (res, rounded)

def _round_sdp_solution(self, sdp_result, table_constructor, block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact, denom=1024):
    r"""
    Round the SDP results output to get something exact.
    """

    debug("\n\nround started\n\n", block_sizes, target_vector_exact, phi_vectors_exact, positives_matrix_exact, denom)
    
    phi_vector_exact = phi_vectors_exact[0] if len(phi_vectors_exact)!=0 else vector([0]*positives_matrix_exact.ncols())

    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints
    
    flags_num = -block_sizes[-2] # same as |F_n|
    
    c_vector_approx = vector(sdp_result['X'][-2]) # dim: |F_n|, c vector, primal slack for flags
    c_vector_rounded = vector(_round_list(c_vector_approx, method=0, denom=denom)) # as above but rounded
    
    # The F (FF) flag indecies where the c vector is zero/nonzero
    c_zero_inds = [FF for FF, xx in enumerate(c_vector_approx) if (abs(xx)<1e-6 or phi_vector_exact[FF]!=0)]
    c_nonzero_inds = [FF for FF in range(flags_num) if FF not in c_zero_inds]
    
    
    
    positives_num = -block_sizes[-1] - 2 # same as m, number of positive constraints (-2 for the equality)

    phi_pos_vector_exact = positives_matrix_exact*phi_vector_exact # dim: m, witness that phi is positive

    e_vector_approx = vector(sdp_result['X'][-1][:-2]) # dim: m, the e vector, primal slack for positivitives
    e_vector_rounded = vector(_round_list(e_vector_approx, method=0, denom=denom)) # as above but rounded

    # The f (ff) positivity constraints where the e vector is zero/nonzero
    e_zero_inds = [ff for ff, xx in enumerate(e_vector_approx) if (abs(xx)<1e-6 or phi_pos_vector_exact[ff]!=0)]
    e_nonzero_inds = [ff for ff in range(positives_num) if ff not in e_zero_inds]
    
    

    if len(phi_vectors_exact)==0: # the u value, the bound we want to prove
        bound_exact = _round(sdp_result['primal']-1e-4, method=0, denom=denom)
    else:
        bound_exact = target_vector_exact*phi_vector_exact 
    # the constraints for the flags that are exact
    corrected_target_relevant_exact = vector([target_vector_exact[FF] - bound_exact for FF in c_zero_inds])
    # the d^f_F matrix, but only the relevant parts for the rounding
    # so F where c_F = 0 and f where e_f != 0
    positives_matrix_relevant_exact = matrix(QQ, len(e_nonzero_inds), len(c_zero_inds), [[positives_matrix_exact[ff][FF] for FF in c_zero_inds] for ff in e_nonzero_inds])
    # the e vector, but only the nonzero entries
    e_nonzero_list_rounded = [e_vector_rounded[ff] for ff in e_nonzero_inds]

    debug("\n\nparameter setup is done\n\n", flags_num, c_vector_approx, c_vector_rounded, c_zero_inds, positives_num, phi_pos_vector_exact)
    debug(e_vector_approx, e_vector_rounded, e_zero_inds, bound_exact)
    
    # 
    # Flatten the matrices relevant for the rounding
    # 
    # M table transforms to a matrix, (with nondiagonal entries doubled)
    # only the FF index matrices corresponding with tight constraints are used
    # 
    # X transforms to a vector
    # only the semidefinite blocks are used
    # 

    # The relevant entries of M flattened to a matrix this will be indexed by 
    # c_zero_inds and the triples from the types
    M_flat_relevant_matrix_exact = matrix(QQ, len(c_zero_inds), 0, 0, sparse=True)
    X_flat_vector_rounded = [] # The rounded X values flattened to a list
    block_index = 0
    block_info = []
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)

        for plus_index, base in enumerate(table_constructor[params]):
            block_info.append([ftype, base])
            X_approx = sdp_result['X'][block_index + plus_index]
            X_flat_vector_rounded += _round_list(_flatten_matrix(X_approx), method=0, denom=denom)

            M_extra = []

            for FF in c_zero_inds:
                M_FF = table[FF]
                M_extra.append(_flatten_matrix((base * M_FF * base.T).rows(), doubled=True))

            M_flat_relevant_matrix_exact = M_flat_relevant_matrix_exact.augment(matrix(M_extra))
        block_index += len(table_constructor[params])

    
    # 
    # Append the relevant M matrix and the X with the additional values from
    # the positivity constraints. 
    #
    # Then correct the x vector values
    # 

    M_matrix_final = M_flat_relevant_matrix_exact.augment(positives_matrix_relevant_exact.T)
    x_vector_final = vector(X_flat_vector_rounded+e_nonzero_list_rounded)
    

    # correct the values of the x vector, based on the minimal L_2 norm
    x_vector_corr = x_vector_final - M_matrix_final.T * \
    (M_matrix_final * M_matrix_final.T).pseudoinverse() * \
    (M_matrix_final*x_vector_final - corrected_target_relevant_exact)

    
    debug("\n\nbefore after correction\n\n", M_matrix_final, x_vector_final, sdp_result['X'], x_vector_corr)
    #
    # Recover the X matrices and e vector from the corrected x
    #
    
    rounding_successful = True
    
    e_nonzero_vector_corr = x_vector_corr[-len(e_nonzero_inds):]
    e_vector_corr = vector(QQ, positives_num, dict(zip(e_nonzero_inds, e_nonzero_vector_corr)))

    if len(e_vector_corr)>0 and min(e_vector_corr)<0:
        rounding_successful = False
        print("Linear coefficient is negative: {}".format(min(e_vector_corr)))

    X_matrix_corr = []
    for ii, block_dim in enumerate(block_sizes):
        if block_dim<0:
            break
        X_matrix_ii_corr, x_vector_corr = _unflatten_matrix(x_vector_corr, block_dim)
        X_matrix_corr.append(matrix(X_matrix_ii_corr))
        if min(X_matrix_ii_corr.eigenvalues())<0:
            rounding_successful = False
            print("Rounded X matrix is not semidefinite: {}".format(min(X_matrix_ii_corr.eigenvalues())))
    X_matrix_corr.append(e_vector_corr)


    #
    # Verify the bound and semidefiniteness
    #

    block_index = 0
    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_corr

    for params in table_constructor.keys():
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)

        for plus_index, base in enumerate(table_constructor[params]):
            X_flat_vector_corr = vector(_flatten_matrix(X_matrix_corr[block_index + plus_index].rows()))

            for gg, morig in enumerate(table):
                mm = base * morig * base.T
                M_flat_vector_exact = vector(_flatten_matrix((base * morig * base.T).rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*X_flat_vector_corr

        block_index += len(table_constructor[params])
    
    debug("\n\nthe recovered final data\n\n", X_matrix_corr, slacks)

    if rounding_successful:
        print("The exact value after rounding is {}, numerically {}".format(min(slacks), min(slacks).n()))
    else:
        print("The rounding was unsuccessful, otherwise the result would be {}".format(min(slacks).n()))

    return min(slacks), X_matrix_corr

In [23]:
ZZ(round(0.999))/3

1/3

In [21]:
G = GraphTheory
G.exclude(G(5))
res = optimize_problem(G, G(2), 5, certificate=True, exact=True)

Base flags generated, their number is 33
The relevant ftypes are constructed, their number is 5
Block sizes before symmetric/asymmetric change is applied: [8, 8, 8, 8, 6]


done with mult table for Ftype on 1 points with edges=[]: : 5it [00:00, 402.64it/s]                 

Tables constructed
Running sdp without construction. Used block sizes are [4, 4, 6, 2, 6, 2, 4, 4, 6, -33, -2]


CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.0756152e+01 Ad: 7.27e-01 Dobj: -1.7012565e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.1130759e+01 Ad: 9.52e-01 Dobj: -4.3177375e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.6757258e+01 Ad: 9.23e-01 Dobj: -4.6196856e-01 
Iter:  4 Ap: 9.89e-01 Pobj: -4.9879562e+00 Ad: 6.80e-01 Dobj: -4.6429854e-01 
Iter:  5 Ap: 5.64e-01 Pobj: -3.9924488e+00 Ad: 9.30e-01 Dobj: -4.7229838e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.2275641e+00 Ad: 7.13e-01 Dobj: -4.8716600e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -9.1340416e-01 Ad: 7.03e-01 Dobj: -5.5747223e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -8.2581438e-01 Ad: 7.40e-01 Dobj: -6.4451514e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -7.7279354e-01 Ad: 7.69e-01 Dobj: -6.9335846e-01 
Iter: 10 Ap: 1.00e+00 Pobj: -7.5637739e-01 Ad: 8.87e-01 Dobj: -7.3189013e-01 
Iter: 11 Ap: 1.00e+00 Pobj: -7.5052711e-01 Ad: 1.00e+00 Dobj: -7.4791845e-01 
Iter: 12 Ap: 9.90e-01 Pobj: -7.5002816e-01 Ad: 1.00e+

In [17]:
def _flatten_matrix(mat, doubled=False):
    r"""
    Flatten a symmetric matrix, optionally double non-diagonal elements
    """
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def _unflatten_matrix(ls, dim, doubled=False):
    r"""
    Unflatten a symmetric matrix, optionally correct for the doubled non-diagonal elements
    """
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

def _round(value, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a number using either 
    method=0 - simple fixed denominator
    method=1 - continued fractions
    """
    if method==0:
        return QQ(round(value*denom))/QQ(denom)
    else:
        from sage.rings.continued_fraction import continued_fraction
        cf = continued_fraction(value)
        for ii, xx in enumerate(cf.quotients()):
            if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
                if ii>0:
                    return cf.convergent(ii-1)
                return 0
        return cf.value()

def _round_list(ls, force_pos=False, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a list
    """
    if force_pos:
        return [max(_round(xx, method, quotient_bound, denom_bound, denom), 0) for xx in ls]
    else:
        return [_round(xx, method, quotient_bound, denom_bound, denom) for xx in ls]

def _round_matrix(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a matrix
    """
    return matrix(QQ, [_round_list(xx, False, method, quotient_bound, denom_bound, denom) for xx in mat])

def _round_ldl(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a matrix using ldl decomposition
    """
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, _round_matrix(mat_ldl[1]), method, quotient_bound, denom_bound, denom)
    D = diagonal_matrix(QQ, _round_list(mat_ldl[2].diagonal(), True, method, quotient_bound, denom_bound, denom))
    pl = P*L
    return pl*D*pl.T

def _round_adaptive(ls, onevec):
    r"""
    Adaptive rounding based on continued fraction and preserving an inner product
    with `onevec`
    """
    
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([_round(xx, quotient_bound=resol1, denom_bound=resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    return best_vec